In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [4]:
len(data)

174643

In [5]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [6]:
import re

In [7]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
        
    return re.sub(pattern, ' ', line)

In [8]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [9]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [10]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    matches = re.findall(pattern, line)
    for match in matches:
        line = re.sub(match, ' ', line)
    return line

In [11]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

In [71]:
%%time
# generate cleansed_data beforehand
cleansed_data = []
for tweet in data:
    # remove_retweet_prefix 
    line = remove_retweet_prefix(tweet)
    # remove hashtag
    line = remove_hashtag(line)
    # remove @...
    line = remove_at(line)
    # remove url
    line = remove_url(line)
    # remove punctuations except apostrophe
    line = cleanse(line)
    cleansed_data.append(line)

CPU times: user 4.57 s, sys: 17.5 ms, total: 4.59 s
Wall time: 4.59 s


In [72]:
print(len(cleansed_data))
cleansed_data[:5]

174643


["JLo's dress     ",
 "What's making Sofia Vergara's boobs stay like that  Magic  Witchcraft   ",
 ' Kerry Washington is EVERYTHING  Dying over her Miu Miu gown       ',
 'Anne Hathaway has got me living   ',
 "Jennifer Lopez's lace dress  Thoughts   "]

In [15]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [39]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[' '.join(t.orth_ for t in entity).strip()]=[entity.label_]
    return tags

In [83]:
import nltk
# nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

def find_sentiment_score(line, verbose=False):
    sentiment_dict = SentimentIntensityAnalyzer().polarity_scores(sentence)
    if verbose:
        print(sentiment_dict)
    return sentiment_dict['neg'], sentiment_dict['neu'], sentiment_dict['pos']

In [84]:
# test  
texts = ["Great place to be when you are in Bangalore.",
            "The place was being renovated when I visited so the seating was limited.",
            "Loved the ambience, loved the food",
            "The food is delicious but not over the top.",
            "Service - Little slow, probably because too many people.",
            "The place is not easy to locate",
            "Mushroom fried rice was tasty"]
  
for sentence in texts[:2]:
    print(sentence)
    print(find_sentiment_score(line, verbose=True))
    print()

Great place to be when you are in Bangalore.
{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.6249}
(0.0, 0.661, 0.339)

The place was being renovated when I visited so the seating was limited.
{'neg': 0.147, 'neu': 0.853, 'pos': 0.0, 'compound': -0.2263}
(0.147, 0.853, 0.0)



In [93]:
# retrieve hosts from hosts.ipynb
# retrieve nominees from nominees.ipynb
# retrieve winners from winners.ipynb
# retrieve presenters from presenters.ipynb


# example
subjects = ['Les Miserables', 'Tina Fey', 'Jay Leno']

In [94]:
def calculate_compound_sentiment(subject, verbose=False):
    pattern = re.compile(r'{0}'.format(subject), re.IGNORECASE)
    neg = 0
    neu = 0
    pos = 0
    num = 0
    for line in cleansed_data:
        match = re.search(pattern, line.lower())
        if match:
            if verbose:
                # print the first 10 occurrences
                if num < 10:
                    print(tweet)
                    print(line)
                    print(tags)
                    print()

            neg_, neu_, pos_ = find_sentiment_score(line)
            neg += neg_
            neu += neu_
            pos += pos_
            num += 1
            
    
    print('subject:', subject)
    print('num of matches:', num)
    print('negative: {0:.4f} \t neutral: {1:.4f} \t positive: {2:.4f}'.format(neg, neu, pos))

In [95]:
%%time
# speed may be a big problem for this part
# need to compare its performance with textblob
for subject in subjects:
    calculate_compound_sentiment(subject, verbose=False)
    print()

subject: Les Miserables
num of matches: 2539
negative: 373.2330 	 neutral: 2165.7670 	 positive: 0.0000

subject: Tina Fey
num of matches: 5847
negative: 859.5090 	 neutral: 4987.4910 	 positive: 0.0000

subject: Jay Leno
num of matches: 171
negative: 25.1370 	 neutral: 145.8630 	 positive: 0.0000

CPU times: user 54.5 s, sys: 62.8 ms, total: 54.5 s
Wall time: 54.6 s


In [36]:
# test
from spacy.tokens import Doc
def polarity_scores(doc):
    return SentimentIntensityAnalyzer().polarity_scores(doc.text) 
Doc.set_extension('polarity_scores', getter=polarity_scores)
 
text = nlp("Really Whaaat event apple nice! it!")
print(text._.polarity_scores)

{'neg': 0.0, 'neu': 0.596, 'pos': 0.404, 'compound': 0.5242}
